In [15]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

gpg: keybox '/tmp/tmp7qma7ayj/pubring.gpg' created
gpg: /tmp/tmp7qma7ayj/trustdb.gpg: trustdb created
gpg: key AD5F235DF639B041: public key "Launchpad PPA for Alessandro Strada" imported
gpg: Total number processed: 1
gpg:               imported: 1
··········


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
import os
os.chdir("drive/Colab Notebooks/LSTM/")

In [19]:
!ls

drive  LSTM.ipynb  macbeth.txt


In [0]:
!pip install -q keras

In [0]:
import numpy 
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import np_utils

In [0]:
filename = 'macbeth.txt'
text = (open(filename).read()).lower()

unique_chars = sorted(list(set(text)))

char_to_int ={}

int_to_char = {}

for i, c in enumerate(unique_chars):
  char_to_int.update({c: i})
  int_to_char.update({i: c})

In [23]:
char_to_int

{'\n': 0,
 ' ': 1,
 '!': 2,
 '&': 3,
 "'": 4,
 '(': 5,
 ')': 6,
 ',': 7,
 '-': 8,
 '.': 9,
 '1': 10,
 '2': 11,
 '3': 12,
 ':': 13,
 ';': 14,
 '?': 15,
 '[': 16,
 ']': 17,
 'a': 18,
 'b': 19,
 'c': 20,
 'd': 21,
 'e': 22,
 'f': 23,
 'g': 24,
 'h': 25,
 'i': 26,
 'k': 27,
 'l': 28,
 'm': 29,
 'n': 30,
 'o': 31,
 'p': 32,
 'q': 33,
 'r': 34,
 's': 35,
 't': 36,
 'u': 37,
 'v': 38,
 'w': 39,
 'x': 40,
 'y': 41,
 'z': 42}

In [0]:
X = []
Y = []

for i in range(0, len(text) - 50, 1):
  sequence = text[i: i + 50]
  label = text[i + 50]
  X.append([char_to_int[char] for char in sequence])
  Y.append(char_to_int[label])

In [25]:
X[0]

[36,
 25,
 22,
 1,
 36,
 34,
 18,
 24,
 22,
 21,
 26,
 22,
 1,
 31,
 23,
 1,
 29,
 18,
 20,
 19,
 22,
 36,
 25,
 0,
 0,
 18,
 20,
 36,
 37,
 35,
 1,
 32,
 34,
 26,
 29,
 37,
 35,
 9,
 1,
 35,
 20,
 31,
 22,
 30,
 18,
 1,
 32,
 34,
 26,
 29]

In [0]:
X_modified = numpy.reshape(X, (len(X), 50, 1))
X_modified = X_modified / float(len(unique_chars))
Y_modified = np_utils.to_categorical(Y)

In [0]:
model = Sequential()
model.add(LSTM(300, input_shape = (X_modified.shape[1], X_modified.shape[2]), return_sequences = True))
model.add(Dropout(0.2))
model.add(LSTM(300))
model.add(Dropout(0.2))
model.add(Dense(Y_modified.shape[1], activation = 'softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [28]:
model.fit(X_modified, Y_modified, epochs=1, batch_size=30)

Epoch 1/1
100270/100270 [==============================] - 800s 8ms/step - loss: 2.7499


In [29]:
start_index = numpy.random.randint(0, len(X) - 1)
new_string = X[start_index]
for i in range(50):
  x = numpy.reshape(new_string, (1, len(new_string), 1))
  x = x / float(len(unique_chars))
  
pred_index = numpy.argmax(model.predict(x, verbose = 0))
char_out = int_to_char[pred_index]
seq_in = [int_to_char[value] for value in new_string]
print (char_out)

e


In [0]:
new_string.append(pred_index)
new_string = new_string[1:len(new_string)]

In [33]:
pred_index

22

In [32]:
new_string

[1,
 35,
 36,
 34,
 31,
 18,
 27,
 22,
 35,
 1,
 38,
 32,
 31,
 30,
 1,
 36,
 25,
 22,
 1,
 23,
 31,
 22,
 13,
 0,
 22,
 40,
 20,
 22,
 32,
 36,
 1,
 36,
 25,
 22,
 41,
 1,
 29,
 22,
 18,
 30,
 36,
 1,
 36,
 31,
 1,
 19,
 18,
 36,
 25,
 22]